# 04 — PointNet v5 Training (resume from v4, extended)

**Changes vs v4:**
- Resume from `best_model_v4.pt` (epoch 132, obs mIoU=0.205)
- **300 total epochs** (+150 from v4)
- More aggressive drop augmentation (0-50% vs 0-30%) for density robustness
- Lower LR restart (2e-4 vs 5e-4) for fine-tuning
- Same model architecture (1.88M params)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q onnx onnxscript

In [ ]:
import os, sys, gc, time, json
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

DRIVE_BASE = "/content/drive/MyDrive/airbus_hackathon"
DATA_DIR = f"{DRIVE_BASE}/data"
CKPT_V4_DIR = f"{DRIVE_BASE}/checkpoints_v4"
CKPT_DIR = f"{DRIVE_BASE}/checkpoints_v5"
os.makedirs(CKPT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram:.1f} GB")

## 1. Config

In [ ]:
# === CONFIG v5 (resume from v4) ===
SCENE_FILES = [f"scene_{i}.h5" for i in range(1, 11)]
VAL_SCENE = "scene_8"
NUM_CLASSES = 5
IN_CHANNELS = 5  # x, y, z, reflectivity, norm_distance
CLASS_NAMES = {0: "background", 1: "antenna", 2: "cable", 3: "electric_pole", 4: "wind_turbine"}
OBSTACLE_CLASSES = [1, 2, 3, 4]
CLASS_COLORS = {
    (38, 23, 180): 1, (177, 132, 47): 2,
    (129, 81, 97): 3, (66, 132, 9): 4,
}

# === GPU resources ===
NUM_POINTS = 65536
BATCH_SIZE = 16
OBSTACLE_RATIO = 0.5

# === TRAINING v5 — extended from v4 ===
TOTAL_EPOCHS = 300      # v4 did 150, we continue to 300
V4_EPOCHS = 150         # v4 trained this many
NEW_EPOCHS = TOTAL_EPOCHS - V4_EPOCHS  # 150 more
LR = 2e-4               # lower than v4 (5e-4) for fine-tuning
WARMUP_EPOCHS = 3       # short warmup for resume
MAX_GRAD_NORM = 1.0

# Focal loss (same as v4)
FOCAL_ALPHA = [0.10, 0.20, 0.30, 0.25, 0.15]
FOCAL_GAMMA = 2.0

# Augmentation — more aggressive drop for density robustness
DROP_RANGE = (0.0, 0.50)  # was (0.0, 0.30) in v4

print(f"Config v5: resume v4 + {NEW_EPOCHS} epochs (total {TOTAL_EPOCHS})")
print(f"LR={LR} (fine-tuning), drop augment up to {DROP_RANGE[1]*100:.0f}%")

## 2. Dataset (same as v4, more aggressive drop)

In [ ]:
def get_frame_boundaries(h5_path, dataset_name="lidar_points", chunk_size=2_000_000):
    """Find frame boundaries by reading in chunks."""
    change_indices = []
    with h5py.File(h5_path, "r") as f:
        ds = f[dataset_name]
        n = ds.shape[0]
        prev_last_pose = None
        for offset in range(0, n, chunk_size):
            end = min(offset + chunk_size, n)
            chunk = ds[offset:end]
            ex, ey, ez, eyaw = chunk["ego_x"], chunk["ego_y"], chunk["ego_z"], chunk["ego_yaw"]
            if prev_last_pose is not None:
                cur_first = (int(ex[0]), int(ey[0]), int(ez[0]), int(eyaw[0]))
                if cur_first != prev_last_pose:
                    change_indices.append(offset)
            changes = np.where(
                (np.diff(ex) != 0) | (np.diff(ey) != 0) |
                (np.diff(ez) != 0) | (np.diff(eyaw) != 0)
            )[0] + 1
            for c in changes:
                change_indices.append(offset + int(c))
            prev_last_pose = (int(ex[-1]), int(ey[-1]), int(ez[-1]), int(eyaw[-1]))
            del chunk, ex, ey, ez, eyaw
    starts = [0] + change_indices
    ends = change_indices + [n]
    frames = []
    with h5py.File(h5_path, "r") as f:
        ds = f[dataset_name]
        for s, e in zip(starts, ends):
            row = ds[s]
            frames.append((s, e, int(row["ego_x"]), int(row["ego_y"]),
                           int(row["ego_z"]), int(row["ego_yaw"])))
    return frames


def map_rgb_to_class(r, g, b):
    class_ids = np.zeros(len(r), dtype=np.int64)
    for (cr, cg, cb), cid in CLASS_COLORS.items():
        mask = (r == cr) & (g == cg) & (b == cb)
        class_ids[mask] = cid
    return class_ids


class LidarSegDatasetV5(Dataset):
    """Same as v4 with more aggressive drop augmentation."""

    def __init__(self, data_dir, scene_files, num_points=65536,
                 obstacle_ratio=0.5, augment=False, drop_range=(0.0, 0.50)):
        self.data_dir = data_dir
        self.num_points = num_points
        self.obstacle_ratio = obstacle_ratio
        self.augment = augment
        self.drop_range = drop_range
        self.index = []
        for sf in scene_files:
            h5_path = os.path.join(data_dir, sf)
            scene_name = sf.replace(".h5", "")
            if not os.path.exists(h5_path):
                continue
            frames = get_frame_boundaries(h5_path)
            for idx, (start, end, ex, ey, ez, eyaw) in enumerate(frames):
                self.index.append((h5_path, start, end, ex, ey, ez, eyaw, scene_name, idx))
        print(f"LidarSegDatasetV5: {len(self.index)} frames, "
              f"{num_points} pts, drop={drop_range}")

    def __len__(self):
        return len(self.index)

    def _convert_to_features(self, valid):
        dist_m = valid["distance_cm"].astype(np.float64) / 100.0
        az_rad = np.radians(valid["azimuth_raw"].astype(np.float64) / 100.0)
        el_rad = np.radians(valid["elevation_raw"].astype(np.float64) / 100.0)
        cos_el = np.cos(el_rad)
        x = dist_m * cos_el * np.cos(az_rad)
        y = -dist_m * cos_el * np.sin(az_rad)
        z = dist_m * np.sin(el_rad)
        xyz = np.column_stack((x, y, z)).astype(np.float32)
        dist_norm = (dist_m / 300.0).astype(np.float32).reshape(-1, 1)
        refl = (valid["reflectivity"].astype(np.float32) / 255.0).reshape(-1, 1)
        labels = map_rgb_to_class(
            valid["r"].astype(np.uint8),
            valid["g"].astype(np.uint8),
            valid["b"].astype(np.uint8)
        )
        del dist_m, az_rad, el_rad, cos_el, x, y, z
        return xyz, dist_norm, refl, labels

    def _class_balanced_sample(self, xyz, dist_norm, refl, labels):
        n_bg_target = int(self.num_points * (1 - self.obstacle_ratio))
        n_obs_target = self.num_points - n_bg_target
        bg_idx = np.where(labels == 0)[0]
        present_classes = []
        class_indices = {}
        for c in OBSTACLE_CLASSES:
            c_idx = np.where(labels == c)[0]
            if len(c_idx) > 0:
                present_classes.append(c)
                class_indices[c] = c_idx
        if len(present_classes) == 0 or len(bg_idx) == 0:
            n_pts = len(labels)
            replace = n_pts < self.num_points
            idx = np.random.choice(n_pts, self.num_points, replace=replace)
            return xyz[idx], dist_norm[idx], refl[idx], labels[idx]
        n_per_class = n_obs_target // len(present_classes)
        remainder = n_obs_target - n_per_class * len(present_classes)
        obs_selections = []
        for i, c in enumerate(present_classes):
            c_idx = class_indices[c]
            n_want = n_per_class + (1 if i < remainder else 0)
            replace = len(c_idx) < n_want
            sel = np.random.choice(c_idx, n_want, replace=replace)
            obs_selections.append(sel)
        sel_obs = np.concatenate(obs_selections)
        replace = len(bg_idx) < n_bg_target
        sel_bg = np.random.choice(bg_idx, n_bg_target, replace=replace)
        idx = np.concatenate([sel_obs, sel_bg])
        np.random.shuffle(idx)
        return xyz[idx], dist_norm[idx], refl[idx], labels[idx]

    def __getitem__(self, i):
        h5_path, start, end, ex, ey, ez, eyaw, scene_name, frame_idx = self.index[i]
        with h5py.File(h5_path, "r") as f:
            chunk = f["lidar_points"][start:end]
        valid = chunk[chunk["distance_cm"] > 0]
        del chunk
        n_pts = len(valid)
        if n_pts == 0:
            return (torch.zeros(self.num_points, IN_CHANNELS, dtype=torch.float32),
                    torch.zeros(self.num_points, dtype=torch.int64))
        xyz, dist_norm, refl, labels = self._convert_to_features(valid)
        del valid
        xyz, dist_norm, refl, labels = self._class_balanced_sample(xyz, dist_norm, refl, labels)
        if self.augment:
            # Random Z-rotation
            theta = np.random.uniform(0, 2 * np.pi)
            c, s = np.cos(theta), np.sin(theta)
            rot = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]], dtype=np.float32)
            xyz = xyz @ rot.T
            # Jitter
            xyz += np.random.normal(0, 0.02, xyz.shape).astype(np.float32)
            # Random scale
            scale = np.random.uniform(0.9, 1.1)
            xyz *= scale
            # Random flip X or Y
            if np.random.random() > 0.5:
                xyz[:, 0] *= -1
            if np.random.random() > 0.5:
                xyz[:, 1] *= -1
            # Random point drop — MORE AGGRESSIVE in v5
            drop = np.random.uniform(*self.drop_range)
            n_drop = int(self.num_points * drop)
            if n_drop > 0:
                keep = np.random.choice(self.num_points, self.num_points - n_drop, replace=False)
                fill = np.random.choice(keep, n_drop, replace=True)
                order = np.concatenate([keep, fill])
                np.random.shuffle(order)
                xyz = xyz[order]; dist_norm = dist_norm[order]
                refl = refl[order]; labels = labels[order]
        features = np.concatenate([xyz, refl, dist_norm], axis=1)
        return torch.from_numpy(features), torch.from_numpy(labels)

print("Dataset v5 defined.")

## 3. Model + Loss (same as v4)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma=2.0):
        super().__init__()
        self.register_buffer('alpha', torch.tensor(alpha, dtype=torch.float32))
        self.gamma = gamma
    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, reduction='none')
        pt = torch.exp(-ce)
        alpha_t = self.alpha[targets]
        loss = alpha_t * (1 - pt) ** self.gamma * ce
        return loss.mean()


class SharedMLP(nn.Module):
    def __init__(self, in_ch, out_ch, bn=True):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, 1, bias=not bn)
        self.bn = nn.BatchNorm1d(out_ch) if bn else None
    def forward(self, x):
        x = self.conv(x)
        if self.bn: x = self.bn(x)
        return F.relu(x, inplace=True)


class PointNetSegV4(nn.Module):
    """Same architecture as v4 — 1.88M params."""
    def __init__(self, in_channels=5, num_classes=5):
        super().__init__()
        self.enc1 = SharedMLP(in_channels, 64)
        self.enc2 = SharedMLP(64, 128)
        self.enc3 = SharedMLP(128, 256)
        self.enc4 = SharedMLP(256, 512)
        self.enc5 = SharedMLP(512, 1024)
        self.seg1 = SharedMLP(64 + 128 + 256 + 512 + 1024, 512)
        self.seg2 = SharedMLP(512, 256)
        self.seg3 = SharedMLP(256, 128)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.head = nn.Conv1d(128, num_classes, 1)

    def forward(self, x):
        B, N, _ = x.shape
        x = x.transpose(1, 2)
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        e5 = self.enc5(e4)
        g = e5.max(dim=2, keepdim=True)[0].expand(-1, -1, N)
        seg = torch.cat([e1, e2, e3, e4, g], dim=1)
        seg = self.seg1(seg)
        seg = self.dropout1(seg)
        seg = self.seg2(seg)
        seg = self.dropout2(seg)
        seg = self.seg3(seg)
        seg = self.head(seg)
        return seg.transpose(1, 2)


# Load v4 checkpoint
model = PointNetSegV4(in_channels=IN_CHANNELS, num_classes=NUM_CLASSES).to(device)
v4_ckpt_path = os.path.join(CKPT_V4_DIR, "best_model_v4.pt")
print(f"Loading v4 checkpoint: {v4_ckpt_path}")
v4_ckpt = torch.load(v4_ckpt_path, map_location=device, weights_only=False)
model.load_state_dict(v4_ckpt["model_state_dict"])

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"PointNet v4→v5: {n_params:,} params on {device}")
print(f"v4 best: epoch {v4_ckpt['epoch']}, obs mIoU={v4_ckpt['val_obstacle_miou']:.4f}")

## 4. Load Data

In [ ]:
%%time

print("Loading dataset (indexing frame boundaries)...")
train_dataset = LidarSegDatasetV5(DATA_DIR, SCENE_FILES, num_points=NUM_POINTS,
                                   obstacle_ratio=OBSTACLE_RATIO, augment=True,
                                   drop_range=DROP_RANGE)
val_dataset = LidarSegDatasetV5(DATA_DIR, SCENE_FILES, num_points=NUM_POINTS,
                                 obstacle_ratio=OBSTACLE_RATIO, augment=False)

train_idx = [i for i, e in enumerate(train_dataset.index) if e[7] != VAL_SCENE]
val_idx = [i for i, e in enumerate(val_dataset.index) if e[7] == VAL_SCENE]

train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(val_dataset, val_idx)

print(f"Train: {len(train_subset)}, Val: {len(val_subset)}")

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=2, pin_memory=True)

## 5. Training Loop (resume)

In [ ]:
def compute_metrics(preds, labels):
    metrics = {}
    metrics["accuracy"] = (preds == labels).sum().item() / labels.numel()
    obstacle_ious = []
    all_ious = []
    for c in range(NUM_CLASSES):
        intersection = ((preds == c) & (labels == c)).sum().item()
        union = ((preds == c) | (labels == c)).sum().item()
        iou = intersection / union if union > 0 else float("nan")
        metrics[f"iou_{CLASS_NAMES[c]}"] = iou
        if union > 0:
            all_ious.append(iou)
        if c in OBSTACLE_CLASSES and union > 0:
            obstacle_ious.append(iou)
    metrics["obstacle_miou"] = np.nanmean(obstacle_ious) if obstacle_ious else 0.0
    metrics["mean_iou"] = np.nanmean(all_ious) if all_ious else 0.0
    return metrics


def train_one_epoch(model, loader, optimizer, criterion, max_grad_norm):
    model.train()
    total_loss, total_correct, total_pts = 0, 0, 0
    for features, labels in loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(features)
        loss = criterion(logits.reshape(-1, NUM_CLASSES), labels.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        total_loss += loss.item() * features.size(0)
        total_correct += (logits.argmax(-1) == labels).sum().item()
        total_pts += labels.numel()
    return total_loss / len(loader.dataset), total_correct / total_pts


@torch.no_grad()
def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    for features, labels in loader:
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = criterion(logits.reshape(-1, NUM_CLASSES), labels.reshape(-1))
        total_loss += loss.item() * features.size(0)
        all_preds.append(logits.argmax(-1).cpu())
        all_labels.append(labels.cpu())
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    metrics = compute_metrics(all_preds, all_labels)
    metrics["loss"] = total_loss / len(loader.dataset)
    return metrics

print("Training functions defined.")

In [ ]:
# === TRAINING v5 (resume from v4) ===
criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

# Cosine with warm restarts: cycle=75 epochs (half of 150 new epochs)
def lr_lambda(epoch):
    if epoch < WARMUP_EPOCHS:
        return (epoch + 1) / WARMUP_EPOCHS
    cycle_len = 75
    e = epoch - WARMUP_EPOCHS
    cycle_pos = e % cycle_len
    progress = cycle_pos / cycle_len
    return 0.5 * (1 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# Start from v4 best
best_obs_miou = v4_ckpt["val_obstacle_miou"]
history = []

# Load v4 history if available
v4_history_path = os.path.join(CKPT_V4_DIR, "training_history_v4.json")
if os.path.exists(v4_history_path):
    with open(v4_history_path) as f:
        v4_history = json.load(f)
    print(f"Loaded v4 history: {len(v4_history)} epochs")
else:
    v4_history = []

print(f"Resuming from v4 best obs mIoU={best_obs_miou:.4f}")
print(f"Training {NEW_EPOCHS} more epochs (total epochs {V4_EPOCHS+1}..{TOTAL_EPOCHS})")
print(f"Focal Loss (gamma={FOCAL_GAMMA}), LR={LR}, warmup={WARMUP_EPOCHS}")
print(f"Schedule: cosine warm restart (cycle=75 epochs)")
print(f"Drop augmentation: {DROP_RANGE[0]*100:.0f}-{DROP_RANGE[1]*100:.0f}%")
print()
print(f"{'Ep':>3} | {'TrLoss':>7} {'TrAcc':>6} | {'VaLoss':>7} {'ObmIoU':>6} | "
      f"{'Ant':>5} {'Cab':>5} {'Pol':>5} {'Tur':>5} | {'BG':>5} | {'LR':>8} {'T':>4}")
print("-" * 95)

t_total = time.time()

for epoch_offset in range(1, NEW_EPOCHS + 1):
    epoch = V4_EPOCHS + epoch_offset  # global epoch number
    t0 = time.time()

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, MAX_GRAD_NORM)
    val_m = validate(model, val_loader, criterion)
    scheduler.step()

    elapsed = time.time() - t0
    lr = optimizer.param_groups[0]["lr"]

    log = {
        "epoch": epoch, "train_loss": train_loss, "train_acc": train_acc,
        "val_loss": val_m["loss"], "val_acc": val_m["accuracy"],
        "val_obstacle_miou": val_m["obstacle_miou"],
        "val_miou": val_m["mean_iou"], "lr": lr, "time_s": elapsed,
    }
    for c in range(NUM_CLASSES):
        log[f"val_iou_{CLASS_NAMES[c]}"] = val_m.get(f"iou_{CLASS_NAMES[c]}", 0)
    history.append(log)

    ant = val_m.get("iou_antenna", 0) or 0
    cab = val_m.get("iou_cable", 0) or 0
    pol = val_m.get("iou_electric_pole", 0) or 0
    tur = val_m.get("iou_wind_turbine", 0) or 0
    bg = val_m.get("iou_background", 0) or 0

    marker = ""
    if val_m["obstacle_miou"] > best_obs_miou:
        best_obs_miou = val_m["obstacle_miou"]
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "val_obstacle_miou": best_obs_miou,
            "val_metrics": val_m,
            "n_params": n_params,
        }, os.path.join(CKPT_DIR, "best_model_v5.pt"))
        marker = " ** BEST"

    print(f"{epoch:3d} | {train_loss:7.4f} {train_acc:6.4f} | "
          f"{val_m['loss']:7.4f} {val_m['obstacle_miou']:6.4f} | "
          f"{ant:5.3f} {cab:5.3f} {pol:5.3f} {tur:5.3f} | "
          f"{bg:5.3f} | {lr:8.6f} {elapsed:4.0f}s{marker}")

    # Checkpoint every 25 epochs
    if epoch_offset % 25 == 0:
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
        }, os.path.join(CKPT_DIR, f"checkpoint_v5_epoch{epoch}.pt"))

    if epoch_offset == 1 and torch.cuda.is_available():
        vram_used = torch.cuda.max_memory_allocated() / 1e9
        print(f"  [VRAM peak: {vram_used:.1f} GB / {vram:.1f} GB ({vram_used/vram*100:.0f}%)]")

total_time = time.time() - t_total
print(f"\n{'='*60}")
print(f"TRAINING v5 COMPLETE in {total_time:.0f}s ({total_time/60:.1f} min)")
print(f"Best obstacle mIoU: {best_obs_miou:.4f}")
print(f"Model: {n_params:,} params")

## 6. Save History & ONNX Export

In [ ]:
# Merge v4 + v5 history
full_history = v4_history + history
with open(os.path.join(CKPT_DIR, "training_history_v5.json"), "w") as f:
    json.dump(full_history, f, indent=2)
print(f"Full history saved: {len(full_history)} epochs")

# Load best v5 model and export ONNX
best_v5_path = os.path.join(CKPT_DIR, "best_model_v5.pt")
if os.path.exists(best_v5_path):
    best_ckpt = torch.load(best_v5_path, map_location=device, weights_only=False)
    model.load_state_dict(best_ckpt["model_state_dict"])
    print(f"Best v5: epoch {best_ckpt['epoch']}, obs mIoU={best_ckpt['val_obstacle_miou']:.4f}")
else:
    print("No improvement over v4 — using v4 best")
    best_ckpt = v4_ckpt

model.eval()
dummy_input = torch.randn(1, NUM_POINTS, IN_CHANNELS, device=device)
onnx_path = os.path.join(CKPT_DIR, "pointnet_lite_v5.onnx")

torch.onnx.export(
    model, dummy_input, onnx_path,
    input_names=["points"],
    output_names=["logits"],
    dynamic_axes={"points": {0: "batch", 1: "num_points"},
                  "logits": {0: "batch", 1: "num_points"}},
    opset_version=17,
)

onnx_size = os.path.getsize(onnx_path) / 1e6
print(f"ONNX exported: {onnx_size:.2f} MB")

## 7. Training Curves (v4+v5 combined)

In [ ]:
import matplotlib.pyplot as plt

epochs_list = [h["epoch"] for h in full_history]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss
axes[0,0].plot(epochs_list, [h["train_loss"] for h in full_history], label="Train")
axes[0,0].plot(epochs_list, [h["val_loss"] for h in full_history], label="Val")
axes[0,0].axvline(x=V4_EPOCHS, color='gray', linestyle='--', alpha=0.5, label='v4→v5')
axes[0,0].set_xlabel("Epoch"); axes[0,0].set_ylabel("Loss")
axes[0,0].set_title("Focal Loss"); axes[0,0].legend()

# Obstacle mIoU
axes[0,1].plot(epochs_list, [h["val_obstacle_miou"] for h in full_history], 'g-', linewidth=2, label="Obstacle mIoU")
axes[0,1].plot(epochs_list, [h.get("val_miou", h.get("val_obstacle_miou", 0)) for h in full_history], 'b--', alpha=0.5, label="All mIoU")
axes[0,1].axvline(x=V4_EPOCHS, color='gray', linestyle='--', alpha=0.5, label='v4→v5')
axes[0,1].axhline(y=0.205, color='r', linestyle=':', alpha=0.5, label='v4 best (0.205)')
axes[0,1].set_xlabel("Epoch"); axes[0,1].set_ylabel("mIoU")
axes[0,1].set_title(f"Val mIoU (best obs={best_obs_miou:.4f})"); axes[0,1].legend()

# Per-class IoU
colors = {'antenna': 'blue', 'cable': 'orange', 'electric_pole': 'green', 'wind_turbine': 'red'}
for c in range(1, 5):
    name = CLASS_NAMES[c]
    vals = [h.get(f"val_iou_{name}", 0) or 0 for h in full_history]
    axes[1,0].plot(epochs_list, vals, label=name, color=colors[name])
axes[1,0].axvline(x=V4_EPOCHS, color='gray', linestyle='--', alpha=0.5)
axes[1,0].set_xlabel("Epoch"); axes[1,0].set_ylabel("IoU")
axes[1,0].set_title("Per-class IoU (obstacles)"); axes[1,0].legend()

# LR schedule
axes[1,1].plot(epochs_list, [h["lr"] for h in full_history], 'purple')
axes[1,1].axvline(x=V4_EPOCHS, color='gray', linestyle='--', alpha=0.5, label='v4→v5')
axes[1,1].set_xlabel("Epoch"); axes[1,1].set_ylabel("LR")
axes[1,1].set_title("Learning Rate"); axes[1,1].legend()

plt.tight_layout()
plt.savefig(os.path.join(CKPT_DIR, "training_curves_v5.png"), dpi=150)
plt.show()
print("Curves saved.")

## 8. Summary

In [ ]:
print("=" * 60)
print("TRAINING SUMMARY (v5 — resumed from v4)")
print("=" * 60)
print(f"Model: PointNet v4 architecture ({n_params:,} params)")
print(f"v4 best: epoch {v4_ckpt['epoch']}, obs mIoU={v4_ckpt['val_obstacle_miou']:.4f}")
print(f"v5 best: obs mIoU={best_obs_miou:.4f}")
improvement = best_obs_miou - v4_ckpt['val_obstacle_miou']
print(f"Improvement: {improvement:+.4f} ({improvement/v4_ckpt['val_obstacle_miou']*100:+.1f}%)")
print(f"\nv5 changes: LR={LR}, drop augment {DROP_RANGE}, {NEW_EPOCHS} more epochs")
print(f"Total training time (v5 only): {total_time/60:.1f} min")

if os.path.exists(best_v5_path):
    best_ckpt = torch.load(best_v5_path, map_location='cpu', weights_only=False)
    print(f"\nPer-class IoU (best v5 epoch {best_ckpt['epoch']}):")
    for c in range(NUM_CLASSES):
        iou = best_ckpt['val_metrics'].get(f'iou_{CLASS_NAMES[c]}', 0)
        if iou != iou: iou = 0
        v4_iou = v4_ckpt['val_metrics'].get(f'iou_{CLASS_NAMES[c]}', 0)
        if v4_iou != v4_iou: v4_iou = 0
        delta = iou - v4_iou
        print(f"  {CLASS_NAMES[c]:15s}: {iou:.4f} (v4: {v4_iou:.4f}, {delta:+.4f})")

print(f"\nProgression v1 -> v5:")
print(f"  v1: obs mIoU ~ 0.05  (weighted CE, random sampling)")
print(f"  v2: obs mIoU ~ 0.03  (focal loss, still unbalanced)")
print(f"  v3: obs mIoU = 0.168 (focal + balanced 50/50)")
print(f"  v4: obs mIoU = 0.205 (class-balanced + bigger model)")
print(f"  v5: obs mIoU = {best_obs_miou:.3f} (resumed + aggressive drop augment)")